# Song Lyrics -> Cleaning and Emotion Analysis with BERT

In [1]:
# pip install --upgrade pandas

In [1]:
import os
import pandas as pd
import string
import tensorflow as tf
import re
import ast
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

2023-06-07 16:35:02.756002: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 16:35:03.607024: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
with open('generatedDS.pkl', 'rb') as file:
    data_lyrics = pd.read_pickle(file)

In [3]:

data_lyrics = data_lyrics.dropna()

data_lyrics

,track,artist,seq
127795,I Hear Music Everywhere,Barney,"Music's always inside my head_x000D_\nYes, all..."
116535,Gravity [*],Sara Bareilles,Something always brings me back to you_x000D_\...
116894,The Christmas Song,Sarah Connor,Chestnuts roasting on an open fire_x000D_\nJac...
21419,Stella Is a Planet,Tripping Daisy,She's the melting pot of summer_x000D_\nShe's ...
40181,Skylark,Roger Kellaway,Skylark_x000D_\nHave you anything to say to me...
...,...,...,...
70670,My Confession,Ray Price,Sweetheart I'm all alone tonight thinking abou...
110785,One,Rob Klopfenstein,I try with all my might but still don't unders...
77529,Dog Eat Dog,Weird Al Yankovic,Found a job in a great big office\nAnd I reall...
129908,The Mystery of Love,Prefab Sprout,Have you ever thought who made all those stars...


In [4]:
data_lyrics = data_lyrics[data_lyrics['seq'].str.strip() != '']
data_lyrics

,track,artist,seq
127795,I Hear Music Everywhere,Barney,"Music's always inside my head_x000D_\nYes, all..."
116535,Gravity [*],Sara Bareilles,Something always brings me back to you_x000D_\...
116894,The Christmas Song,Sarah Connor,Chestnuts roasting on an open fire_x000D_\nJac...
21419,Stella Is a Planet,Tripping Daisy,She's the melting pot of summer_x000D_\nShe's ...
40181,Skylark,Roger Kellaway,Skylark_x000D_\nHave you anything to say to me...
...,...,...,...
70670,My Confession,Ray Price,Sweetheart I'm all alone tonight thinking abou...
110785,One,Rob Klopfenstein,I try with all my might but still don't unders...
77529,Dog Eat Dog,Weird Al Yankovic,Found a job in a great big office\nAnd I reall...
129908,The Mystery of Love,Prefab Sprout,Have you ever thought who made all those stars...


In [5]:
data_lyrics['clean_text'] = data_lyrics['seq'].str.lower().str.translate(str.maketrans('', '', string.punctuation))
data_lyrics

,track,artist,seq,clean_text
127795,I Hear Music Everywhere,Barney,"Music's always inside my head_x000D_\nYes, all...",musics always inside my headx000d\nyes all day...
116535,Gravity [*],Sara Bareilles,Something always brings me back to you_x000D_\...,something always brings me back to youx000d\ni...
116894,The Christmas Song,Sarah Connor,Chestnuts roasting on an open fire_x000D_\nJac...,chestnuts roasting on an open firex000d\njack ...
21419,Stella Is a Planet,Tripping Daisy,She's the melting pot of summer_x000D_\nShe's ...,shes the melting pot of summerx000d\nshes a ba...
40181,Skylark,Roger Kellaway,Skylark_x000D_\nHave you anything to say to me...,skylarkx000d\nhave you anything to say to mex0...
...,...,...,...,...
70670,My Confession,Ray Price,Sweetheart I'm all alone tonight thinking abou...,sweetheart im all alone tonight thinking about...
110785,One,Rob Klopfenstein,I try with all my might but still don't unders...,i try with all my might but still dont underst...
77529,Dog Eat Dog,Weird Al Yankovic,Found a job in a great big office\nAnd I reall...,found a job in a great big office\nand i reall...
129908,The Mystery of Love,Prefab Sprout,Have you ever thought who made all those stars...,have you ever thought who made all those stars...


In [6]:
data_lyrics['clean_text'] = data_lyrics['clean_text'].apply(lambda x: re.sub(r"\n", " ", x))
data_lyrics['clean_text'] = data_lyrics['clean_text'].apply(lambda x: re.sub(r"x[0-9]+d", " ", x))
# data_lyrics['clean_text'] = data_lyrics['clean_text'].apply(lambda x: re.sub(r"_x[0-9]+D_\n", " ", x))
data_lyrics['clean_text'] = data_lyrics['clean_text'].apply(lambda x: re.sub(r"x[0-9]+", " ", x))
data_lyrics['clean_text'] = data_lyrics['clean_text'].apply(lambda x: re.sub(r"\s+", " ", x))
data_lyrics

,track,artist,seq,clean_text
127795,I Hear Music Everywhere,Barney,"Music's always inside my head_x000D_\nYes, all...",musics always inside my head yes all day long ...
116535,Gravity [*],Sara Bareilles,Something always brings me back to you_x000D_\...,something always brings me back to you it neve...
116894,The Christmas Song,Sarah Connor,Chestnuts roasting on an open fire_x000D_\nJac...,chestnuts roasting on an open fire jack frost ...
21419,Stella Is a Planet,Tripping Daisy,She's the melting pot of summer_x000D_\nShe's ...,shes the melting pot of summer shes a balloon ...
40181,Skylark,Roger Kellaway,Skylark_x000D_\nHave you anything to say to me...,skylark have you anything to say to me wont yo...
...,...,...,...,...
70670,My Confession,Ray Price,Sweetheart I'm all alone tonight thinking abou...,sweetheart im all alone tonight thinking about...
110785,One,Rob Klopfenstein,I try with all my might but still don't unders...,i try with all my might but still dont underst...
77529,Dog Eat Dog,Weird Al Yankovic,Found a job in a great big office\nAnd I reall...,found a job in a great big office and i really...
129908,The Mystery of Love,Prefab Sprout,Have you ever thought who made all those stars...,have you ever thought who made all those stars...


In [7]:
#Take only first five hundred characters of each song to reduce text size
data_lyrics['clean_text'] = data_lyrics['clean_text'].str.slice(0, 500)
data_lyrics

,track,artist,seq,clean_text
127795,I Hear Music Everywhere,Barney,"Music's always inside my head_x000D_\nYes, all...",musics always inside my head yes all day long ...
116535,Gravity [*],Sara Bareilles,Something always brings me back to you_x000D_\...,something always brings me back to you it neve...
116894,The Christmas Song,Sarah Connor,Chestnuts roasting on an open fire_x000D_\nJac...,chestnuts roasting on an open fire jack frost ...
21419,Stella Is a Planet,Tripping Daisy,She's the melting pot of summer_x000D_\nShe's ...,shes the melting pot of summer shes a balloon ...
40181,Skylark,Roger Kellaway,Skylark_x000D_\nHave you anything to say to me...,skylark have you anything to say to me wont yo...
...,...,...,...,...
70670,My Confession,Ray Price,Sweetheart I'm all alone tonight thinking abou...,sweetheart im all alone tonight thinking about...
110785,One,Rob Klopfenstein,I try with all my might but still don't unders...,i try with all my might but still dont underst...
77529,Dog Eat Dog,Weird Al Yankovic,Found a job in a great big office\nAnd I reall...,found a job in a great big office and i really...
129908,The Mystery of Love,Prefab Sprout,Have you ever thought who made all those stars...,have you ever thought who made all those stars...


In [8]:
# nltk.download('punkt')

In [9]:
column_name = 'clean_text'

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

data_lyrics[column_name] = data_lyrics[column_name].apply(remove_stopwords)


[nltk_data] Downloading package stopwords to /home/andres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
data_lyrics

,track,artist,seq,clean_text
127795,I Hear Music Everywhere,Barney,"Music's always inside my head_x000D_\nYes, all...",musics always inside head yes day long till go...
116535,Gravity [*],Sara Bareilles,Something always brings me back to you_x000D_\...,something always brings back never takes long ...
116894,The Christmas Song,Sarah Connor,Chestnuts roasting on an open fire_x000D_\nJac...,chestnuts roasting open fire jack frost nippin...
21419,Stella Is a Planet,Tripping Daisy,She's the melting pot of summer_x000D_\nShe's ...,shes melting pot summer shes balloon gentle br...
40181,Skylark,Roger Kellaway,Skylark_x000D_\nHave you anything to say to me...,skylark anything say wont tell love meadow mis...
...,...,...,...,...
70670,My Confession,Ray Price,Sweetheart I'm all alone tonight thinking abou...,sweetheart im alone tonight thinking wasted pa...
110785,One,Rob Klopfenstein,I try with all my might but still don't unders...,try might still dont understand ever let get h...
77529,Dog Eat Dog,Weird Al Yankovic,Found a job in a great big office\nAnd I reall...,found job great big office really love place g...
129908,The Mystery of Love,Prefab Sprout,Have you ever thought who made all those stars...,ever thought made stars endless sky mystery my...


In [12]:
# with open('cleaned_generatedDS.pkl', 'wb') as file:
#     pickle.dump(data_lyrics, file)

# Obtain most important words on each lyric

In [13]:
# pip install -U scikit-learn

In [ ]:


from sklearn.feature_extraction.text import TfidfVectorizer


# Create TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data_lyrics['clean_text'])

n_top_words = 10

top_words_list = []

for i in range(len(data_lyrics)):
    lyric_tfidf_scores = tfidf_matrix[i].toarray()[0]
    top_words_indices = lyric_tfidf_scores.argsort()[-n_top_words:][::-1]
    top_words = [vectorizer.get_feature_names_out()[index] for index in top_words_indices]
    top_words_list.append(top_words)
    # print(f"Top {n_top_words} words for song {i+1}: {top_words}")

data_lyrics['top_words'] = top_words_list

data_lyrics

In [19]:
# data_lyrics.to_excel("tfidfdatalyrics.xlsx")

In [12]:
data_lyrics = pd.read_excel("tfidfdatalyrics.xlsx")
data_lyrics

,Unnamed: 0,track,artist,seq,clean_text,top_words
0,115192,Exactly Like You,Scatman Crothers,I used to have a perfect sweetheart_x000D_\nNo...,used perfect sweetheart real one dream wonderf...,"['exactly', 'ideal', 'real', 'live', 'scenes',..."
1,105636,Too Marvelous for Words,Dick Robertson,"You're just too marvelous, too marvelous for w...",youre marvelous marvelous words like glorious ...,"['marvelous', 'websters', 'dictionary', 'much'..."
2,143992,Skyscrapers [Version],OK Go,"Skyscrapers, please forgive me_x000D_\nI didn'...",skyscrapers please forgive didnt mean word sai...,"['skyscrapers', 'standing', 'forgive', 'repent..."
3,61502,Next Time Around,Emiliana Torrini,Then came question and it was about time_x000D...,came question time answer came back long house...,"['came', 'architect', 'talented', 'dialogue', ..."
4,111565,September Song [From Knickerbocker Holiday],Theodore Bikel,"Oh, it's a long, long while from May to Decemb...",oh long long may december days grow short reac...,"['precious', 'days', 'september', 'spend', 'dw..."
...,...,...,...,...,...,...
14995,70840,Alone in Your Heart,Monica,When them other girls ask you if I'm your girl...,girls ask im girl say ooh ooh ooh diamond pear...,"['aint', 'elses', 'ooh', 'nobody', 'oh', 'seer..."
14996,124981,Too Busy Thinking 'Bout My Baby,Andy Abraham,Listen to me people_x000D_\nI ain't got time t...,listen people aint got time think money buy ai...,"['aint', 'time', 'got', 'studying', 'discuss',..."
14997,156921,The Devil in Georgia,Fiddlin' Doc Roberts,The devil went down to Georgia_x000D_\nHe was ...,devil went georgia lookin soul steal bind caus...,"['fiddle', 'devil', 'bet', 'sawin', 'hickory',..."
14998,154389,Bringin' It,Timbaland,What_x000D_\nYo yo yo yo yo yo_x000D_\nHa ha_x...,yo yo yo yo yo yo ha ha aint baby dont stop li...,"['dat', 'wahkump', 'yo', 'datdat', 'thug', 'ha..."


In [26]:
with open('tfidfdatalyrics.pkl', 'wb') as file:
    pickle.dump(data_lyrics, file)

# Obtain emotion for each song using BERT (Zero-Shot)

In [17]:
# !pip install transformers

In [13]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

/home/andres/miniconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# pip install --upgrade accelerate

In [15]:
# load tokenizer and model, create trainer
model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

In [16]:
pred_texts = data_lyrics['clean_text'].tolist()

In [17]:
tokenized_texts = tokenizer(pred_texts,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

In [19]:
predictions = trainer.predict(pred_dataset)

In [20]:
# Transform predictions to labels
preds = predictions.predictions.argmax(-1)
labels = pd.Series(preds).map(model.config.id2label)
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)

In [21]:
# scores raw
temp = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True))

In [22]:
# work in progress
# container
anger = []
disgust = []
fear = []
joy = []
neutral = []
sadness = []
surprise = []


#    'Amusement': 0,
#     'Anger': 1,
#     'Awe': 2,
#     'Contentment': 3,
#     'Excitement': 4,
#     'Fear': 5,
#     'Sadness': 6,
#     'Something else': 7



# extract scores (as many entries as exist in pred_texts)
for i in range(len(pred_texts)):
  anger.append(temp[i][0])
  disgust.append(temp[i][1])
  fear.append(temp[i][2])
  joy.append(temp[i][3])
  neutral.append(temp[i][4])
  sadness.append(temp[i][5])
  surprise.append(temp[i][6])

In [23]:
# Create DataFrame with texts, predictions, labels, and scores
df = pd.DataFrame(list(zip(pred_texts,preds,labels,scores,  anger, disgust, fear, joy, neutral, sadness, surprise)), columns=['text','pred','label','score', 'anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise'])
# df.to_excel("dataframe_sentimientos.xlsx")
df

,text,pred,label,score,anger,disgust,fear,joy,neutral,sadness,surprise
0,used perfect sweetheart real one dream wonderf...,3,joy,0.876704,0.006888,0.000465,0.001014,0.876704,0.003475,0.091479,0.019975
1,youre marvelous marvelous words like glorious ...,3,joy,0.964882,0.001332,0.000263,0.001152,0.964882,0.004189,0.002530,0.025652
2,skyscrapers please forgive didnt mean word sai...,5,sadness,0.437704,0.391379,0.004439,0.003745,0.075735,0.039086,0.437704,0.047912
3,came question time answer came back long house...,6,surprise,0.472173,0.071424,0.000989,0.028923,0.052858,0.160615,0.213016,0.472173
4,oh long long may december days grow short reac...,5,sadness,0.788969,0.054612,0.000785,0.002915,0.041115,0.040510,0.788969,0.071095
...,...,...,...,...,...,...,...,...,...,...,...
14995,girls ask im girl say ooh ooh ooh diamond pear...,5,sadness,0.606736,0.011321,0.001819,0.004774,0.097600,0.039067,0.606736,0.238683
14996,listen people aint got time think money buy ai...,6,surprise,0.727892,0.018370,0.000586,0.002269,0.083473,0.075361,0.092050,0.727892
14997,devil went georgia lookin soul steal bind caus...,0,anger,0.387126,0.387126,0.001800,0.013905,0.249973,0.061830,0.180453,0.104913
14998,yo yo yo yo yo yo ha ha aint baby dont stop li...,2,fear,0.921688,0.063980,0.001348,0.921688,0.002401,0.004958,0.003350,0.002275


In [24]:
df

,text,pred,label,score,anger,disgust,fear,joy,neutral,sadness,surprise
0,used perfect sweetheart real one dream wonderf...,3,joy,0.876704,0.006888,0.000465,0.001014,0.876704,0.003475,0.091479,0.019975
1,youre marvelous marvelous words like glorious ...,3,joy,0.964882,0.001332,0.000263,0.001152,0.964882,0.004189,0.002530,0.025652
2,skyscrapers please forgive didnt mean word sai...,5,sadness,0.437704,0.391379,0.004439,0.003745,0.075735,0.039086,0.437704,0.047912
3,came question time answer came back long house...,6,surprise,0.472173,0.071424,0.000989,0.028923,0.052858,0.160615,0.213016,0.472173
4,oh long long may december days grow short reac...,5,sadness,0.788969,0.054612,0.000785,0.002915,0.041115,0.040510,0.788969,0.071095
...,...,...,...,...,...,...,...,...,...,...,...
14995,girls ask im girl say ooh ooh ooh diamond pear...,5,sadness,0.606736,0.011321,0.001819,0.004774,0.097600,0.039067,0.606736,0.238683
14996,listen people aint got time think money buy ai...,6,surprise,0.727892,0.018370,0.000586,0.002269,0.083473,0.075361,0.092050,0.727892
14997,devil went georgia lookin soul steal bind caus...,0,anger,0.387126,0.387126,0.001800,0.013905,0.249973,0.061830,0.180453,0.104913
14998,yo yo yo yo yo yo ha ha aint baby dont stop li...,2,fear,0.921688,0.063980,0.001348,0.921688,0.002401,0.004958,0.003350,0.002275


In [25]:
# SAVE TO PKL

with open('dataframe_sentimientos.pkl', 'wb') as file:
    pickle.dump(df, file)